# <span style="color:darkred">Motor Vehicle Collisions in New York City</span>
### <span style="color:darkred">Final Project - Explainer Notebook</span>
---

This report is written in the course 02806 Social Data Analysis, spring 2017, based on the assignment description for the final exam project found on [GitHub](https://github.com/suneman/socialdataanalysis2017/wiki/Final-Project). 

![Why New York City looks like it does](traffic.png)

Source: [Business Insider, fetched April 2017](http://www.businessinsider.com/why-new-york-city-looks-like-it-does-2015-9?r=US&IR=T&IR=T)

## <span style="color:darkred">Motivation</span>

### <span style="color:darkred">What is your dataset?</span>

The dataset chosen for this project is the  **"NYC Motor Vehicle Traffic Collisions"** dataset from the Public Safety section from the website of [New York City OpenData](http://opendata.cityofnewyork.us/). This contains traffic collision data for boroughs: Manhattan, Staten Island, Bronx, Queens and Brooklyn.

Looking into where accidents happen and cross-referencing this information with the traffic density to find the areas most commonly exposed to accidents, could be used in order to *improve the safety of pedestrians* and to *take proper precautions* for the vehicles in the area whether this concerns either more mirror, traffic regulations etc.

### <span style="color:darkred">Why did you choose this/these particular dataset(s)?</span>

We knew that we wanted to work with some sort of OpenData from a large city in the world. Going through different OpenData sources such as [London OpenData](https://data.london.gov.uk/), [SF OpenData](https://data.sfgov.org/), [European OpenData](https://open-data.europa.eu) that what seemed as a lot of different opportunities in chosing data, was in fact not. A lot of the publicly available data was lacking either in size (only a few years) or extremely badly formatted, only with a few features, a lot of missing values etc. What would be nice was if there were several years (5-10) and lots of different features to chose from. This way it would be possible to take some active choices on what to keep or discard and chose what was important for predictions, basic statistics and so on.

Looking into [NYC OpenData](http://opendata.cityofnewyork.us/) and the Public Safety Category, the "Traffic Collisions" dataset was discovered. Here we get information on where in New York different traffic collisions happens, a timestamp for the accident, the involved parties (pedestrian, motorist biker..), whether any involved parties were killed, which type of vehicle, whether the driver was distrated/unattentive -- the contributing factors, geolocations -- a great level of detail to work on and make predictions on. More on the features of the dataset will be discussed below, first we discuss the end user's experience.

### <span style="color:darkred">What was your goal for the end user's experience?</span>

<span style="color:red">Should we write this more as text -- and probably focus more on the death part?</span>

* Explore the data that is freely available to everyone
* Getting to know something new about the traffic in New York
* Learning from the patterns that can be found in the data
* Prevent accident in the future by localizing exposed areas that need saftey improvements.

From this notebook, we will try to extract some of the most informative summary staatistics and the results of our machine learning methods to try and give a better insight into the traffic in New York and when accidents happen. 

### <span style="color:darkred">What data to look into</span>
#### <span style="color:darkred">Deaths</span>
* Where do they happen
* what is the underlying cause
* can this be prevented? 
* What is being done in NYC now? - Read more about [*Vision Zero*](http://www1.nyc.gov/nyc-resources/service/3860/nyc-vision-zero-action-plan)
* Wordcloud of different addresses?



### <span style="color:darkred">Libraries</span>
**Importing needed packages for entire solution**


In [1]:
#import urllib2
import numpy as np
import re
import csv
import io
import pandas as pd
pd.set_option("display.max_rows",5) # show a maximum of 10 rows in dataframe
import geoplotlib as gpl
from geoplotlib.utils import BoundingBox
import calendar
import pylab as pl
import geopy
import json
from collections import Counter 
from operator import itemgetter
from scipy import stats, linalg
from sklearn.model_selection import train_test_split
from sklearn import tree, datasets, svm
from sklearn.cluster import KMeans

# Geopy - location
from geopy.geocoders import Nominatim
geolocator = Nominatim()
from random import randint
from time import sleep

# Plotting with plotly
import plotly 
from IPython.display import Image
# Henriettes plotly API key og brugernavn -- saves plot in cloud
plotly.tools.set_credentials_file(username='frksteenhoff2', api_key='duu8hsfRmuI5rF2EU8o5')
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools

# Functions tailored for this project to minimize code in notebook 
import externalFunctions as ex

# Plotting color variables
bgBorder  = 'rgba(255, 255, 255, 0)'
ticksAxes = 'rgb(107, 107, 107)'
years_of_interest = [2013, 2014, 2015, 2016]
pd.options.mode.chained_assignment = None  # default='warn'

try:
    to_unicode = unicode
except NameError:
    to_unicode = str

#### <span style="color:darkred">Looking at the raw data</span>
In order to discuss the data, some of the rows from the dataset are visualized below such that it is easier to understand some of the decisions made by the group when it comes to preprocessing and cleaning. 

In [2]:
# Importing data using pandas
traffic_data = pd.read_csv('Traffic_data.csv', low_memory=False, usecols=['DATE', 'TIME', 'YEAR', 'MONTH', 'HOUR', 'WEEKDAY', 'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2', 'ON STREET NAME', 'LOCATION', 'LONGITUDE', 'LATITUDE', 'BOROUGH', 'CONTRIBUTING FACTOR VEHICLE 1', 'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST KILLED']) # Data fetched April

# Giving an example of how the data is structured (features etc.)
traffic_data[:5]

,DATE,TIME,BOROUGH,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,YEAR,MONTH,HOUR,WEEKDAY
0,03/31/2017,0:00,NaN,40.645615,-73.90990,"(40.645615, -73.9099)",FOSTER AVENUE,0,0,0,0,Driver Inattention/Distraction,PASSENGER VEHICLE,PASSENGER VEHICLE,2017,3,0,4
1,03/31/2017,0:00,NaN,40.762737,-73.83951,"(40.762737, -73.83951)",NaN,0,0,0,0,Driver Inattention/Distraction,SPORT UTILITY / STATION WAGON,PASSENGER VEHICLE,2017,3,0,4
2,03/31/2017,0:00,BROOKLYN,40.658478,-73.92818,"(40.658478, -73.92818)",EAST 53 STREET,0,0,0,0,Failure to Yield Right-of-Way,SPORT UTILITY / STATION WAGON,NaN,2017,3,0,4
3,03/31/2017,0:00,BROOKLYN,40.580360,-73.96761,"(40.58036, -73.96761)",NEPTUNE AVENUE,0,0,0,0,NaN,NaN,NaN,2017,3,0,4
4,03/31/2017,0:00,NaN,40.845180,-73.91417,"(40.84518, -73.91417)",JEROME AVENUE,0,0,0,0,Following Too Closely,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON,2017,3,0,4


In [3]:
print "-------------- BASIC INFORMATIOM FROM DATASET ON NYC COLLISIONS --------------\n" 
print "Year range:    ", sorted(traffic_data['YEAR'].unique())
print "\nDate range:    " , min(traffic_data["DATE"]), "-", max(traffic_data["DATE"]) 
print "\nDataset size:  ", len(traffic_data), "incidents"
print "\nList of areas: ", traffic_data['BOROUGH'].unique()
print "\nFeatures:      ", list(traffic_data.columns), "\n"
print "------------------------------------------------------------------------------"

-------------- BASIC INFORMATIOM FROM DATASET ON NYC COLLISIONS --------------

Year range:     [2012, 2013, 2014, 2015, 2016, 2017]

Date range:     01/01/2013 - 12/31/2016

Dataset size:   1007130 incidents

List of areas:  [nan 'BROOKLYN' 'BRONX' 'MANHATTAN' 'QUEENS' 'STATEN ISLAND']

Features:       ['DATE', 'TIME', 'BOROUGH', 'LATITUDE', 'LONGITUDE', 'LOCATION', 'ON STREET NAME', 'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST KILLED', 'CONTRIBUTING FACTOR VEHICLE 1', 'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2', 'YEAR', 'MONTH', 'HOUR', 'WEEKDAY'] 

------------------------------------------------------------------------------


## <span style="color:darkred">The initial dataset</span>
The original dataset has 29 columns. For ease of access to data, and to minimize computing time when working on the data, it was decided to extend the dataset with the features ``TIME, HOUR, WEEKDAY, MONTH, YEAR`` from the given timestamps of each incident. The features were calculated once and saved to the original dataset for further use. The calculations take a substantial amount of time and the group found that this was an easier solution than having to run this upon every new session.

In all we have $29$ features and $1.007.310$ different incidents from July 2012 until March 24th 2017 -- the data is soo fresh! In some of our work with the data, we will be excluding year 2012 and 2017, since we only have the first and the two first Quarters 2017 and 2012, respectively. When excluded this will be stated explicitly. 

### <span style="color:darkred">Choices in data cleaning and preprocessing</span> 
#### <span style="color:darkred">Finding relevant features</span>
We have removed the features ..... since they will be oof no use for our work. <span style="color:red">Need to write some more here, Jonas?</span>
* Why include/exclude a feature
* What is relevant/irrelevant
* Plotting different boroughs at different times, years per different vehicles etc.

#### <span style="color:darkred">Handling missing values</span>

##### Are missing values a problem in the dataset?
To answer the questions regarding missing values we will show some exampels from the data. We start by giving an example of a feature, ``LOCATION``, with missing values. Here we assume that a missing ``LONGITUDE`` also means a missing ``LATITUDE`` -- both are assumend calculated on the basis of the feature ``LOCATION`` and since this is the root cause of the problem, this is the feature we choose. For this example we have extracted the missing values for each month in 2016.

In [4]:
allData = pd.DataFrame()
misData = pd.DataFrame()

year = [2016]
feature = "LOCATION"

# Extract data to look at (year 2016)
allData = traffic_data.loc[traffic_data['YEAR'].isin(year)]
# Find all missing values in column ("LONGITUDE")

allData[feature] = allData[feature].replace(np.NaN, 'UNKNOWN')
misData = allData.loc[allData[feature] != 'UNKNOWN']

# Print results (more or less) neatly
print "Accidents in all in NYC, 2016:  ", '{0:,}'.format(len(allData))
print "Accidents with geolocations:    ", '{0:,}'.format(len(misData))
print "% of accidents missing location:", round(100.0-float(len(misData))/float(len(allData))*100, 2), "%\n" 
print "Month:    All values: Complete obs:   Missing location (%):"
# For each month in given year
for numb in sorted(allData['MONTH'].unique()):
    all_vals   = len(allData.loc[allData['MONTH'].isin([numb])])
    nonan_vals = len(misData.loc[misData['MONTH'].isin([numb])])
    print calendar.month_name[numb].upper().ljust(9), str('{0:,}'.format(all_vals)).ljust(10) + "  " + str('{0:,}'.format(nonan_vals)).ljust(12), "   " + str(round(100-float(nonan_vals) / float(all_vals) * 100, 2)) + "%"

Accidents in all in NYC, 2016:   227,658
Accidents with geolocations:     145,529
% of accidents missing location: 36.08 %

Month:    All values: Complete obs:   Missing location (%):
JANUARY   18,099      15,395          14.94%
FEBRUARY  15,983      13,557          15.18%
MARCH     18,520      15,300          17.39%
APRIL     18,331      15,062          17.83%
MAY       20,052      15,311          23.64%
JUNE      19,435      13,974          28.1%
JULY      19,867      12,527          36.95%
AUGUST    19,678      2,496           87.32%
SEPTEMBER 19,491      3,039           84.41%
OCTOBER   19,667      12,367          37.12%
NOVEMBER  19,317      13,226          31.53%
DECEMBER  19,218      13,275          30.92%


Overall, we can see that for the whole dataset, mor than $\frac{1}{3}$ of the location values are missing! Trust us when we say that as it often is with real data, missing values are a general trend. If not, feel free to change the ``YEAR/FEATURE`` used for extracting the data and look at it yourselves.

The above analysis of the missing values for location (``lon/lat``) show that the number of observations with missing values is really high. Here we are only looking into 2016, but from this alone we can see that for each month more than $14\%$ of the values are missing. The $15\%$ itself is not alarming, but $50\%$ of the months, more than $\frac{1}{4}$ of the values are missing! Especially the summer months are lacking information -- here $15\%$ is actually what is registered correctly.   

In the following sections we will comment on the missing values problem for the features relevant to our further work one by one. 

##### How will missing values affect our results?
From the preliminary analysis a set of features was flagged as in need of some work in order to present the proper picture of the NYC traffic. Since we want to look into where accidents happen, why they happen it is important to have as much data as possible. Overall, what we would like is to impute the missing values with correct values where it is possible. This means that if we have the location, but we are missing the street name, we would like to compute the one based on the other. Some of our most important features with many missing values are: ``LOCATION``, ``BOROUGH``, ``ON STREET NAME`` and ``CONRIBUTING FACTORS VEHICLE 1``. These features and the considered methods for imputing the values and the final result will now be discussed. The number of missing values are calculated based on 2016 data since this will be one of te years most commonly used.

##### Contributing factors
Feature(s): ``CONTRIBUTING FACTOR VEHICLE 1``
* Percentage of values missing: $~1\%$

Contributing factors is hard to infer from nothing. One way to work around the problem would be to impute the values with the $mode$ -- the most frequently occuring value. This would mean that the $1\%$ of the observations missing a value would be imputed with ``DRIVER INATTENTION``. However this would falsly inflate the number of accidents with this contributing factor and since we are going to apply some machine learning tools on this feature we would want the data to eb "as pure" as possible. We ended up chosing to exclude the observations where contributing factor was missing, even though this might not be optimal solution.

##### Location information
Features: ``ON STREET NAME``, ``LONGITUDE/LATITUDE/LOCATION``, ``BOROUGH``
* Percentage of values missing ``ON STREET NAME``: $~19\%$
* Percentage of values missing ``LON/LAT/LOC``:    $~36\%$
* Percentage of values missing: ``BOROUGH``:       $~32\%$

*Location (geo-location), address (street name), borough*

All of these columns concern location just on different levels. Coordinates which is the most precise, street name that describe a larger, still well-defined area, and boroughs that look at a larger geographically defined area. As can be seen from the percentage given above, in all of the columns, there are several missing values.  

The reason why we are describing these features together is because they all can be imputed using the same python module, namely ``geopy``. The module matches, ``lon/lat``-pairs to find a street name or street names and city to ``lon/lat``-pairs, which is exactly what we need! Below we have tried to map the number of observations missing the different featured mentioned and give an example of how ``geopy`` works.

In [5]:
# Replce missing borough values with "unspecified"
traffic_data['BOROUGH']        = traffic_data['BOROUGH'].replace(np.NaN, 'Unspecified')
traffic_data['LOCATION']       = traffic_data['LOCATION'].replace(np.NaN, 'Unspecified')
traffic_data['ON STREET NAME'] = traffic_data['ON STREET NAME'].replace(np.NaN, 'Unspecified')

# Extract the unspecified values
missing_bor = traffic_data.loc[traffic_data['BOROUGH']        == 'Unspecified']
missing_loc = traffic_data.loc[traffic_data['LOCATION']       == 'Unspecified']
missing_st  = traffic_data.loc[traffic_data['ON STREET NAME'] == 'Unspecified']

# Number of missing values for each feature in dataset 
len(missing_bor), len(missing_loc), len(missing_st)

(266674, 202343, 168104)

In [6]:
# Find all observations missing borough, location AND street name
missing_bl = missing_bor.loc[missing_bor['LOCATION'] == 'Unspecified']
all_3      = missing_bl.loc[missing_bl['ON STREET NAME'] == 'Unspecified']
len(all_3)

49852

Almost $50,000$ rows are missing both ``location``, ``on street name`` and ``borough``. This is not that much considering that we are working $20$ times as much data.

##### ``geopy`` and how it works
Let's give an example, we have the coordinates, and want street name/borough:

In [54]:
location = geolocator.reverse("40.762737, -73.83951")
location.address

u'Van Wyck Expressway, Queens, Queens County, NYC, New York, 11368, United States of America'

Here, by giving the geolocation for a feature with a missing street address, we can extract the street name from the ``location`` object with the call to ``.address``. If we want to go the other way (finding geolocations from street name) we do the following:

In [55]:
location = geolocator.geocode("Van Wyck Expressway, New York City")
location

Location((40.68718, -73.8080149, 0.0))

So, from geolocation we get a string where we can extract an address and the borough. By giving the street name and city, which will always be New York, we get the geolocations, which is exactly what we want.

However. We have run in to one major problem. The number of times you can call the ``geopy`` API per day is not enough for our purposes, as we have to make an excessive amount of calls, even if we extract both address and borough in the same call. Therefore, we have not been able to impute all our values correctly. Therefore, for the most of our data analysis, missing values have been excluded. 

## <span style="color:darkred">Summary statistics and basic plotting</span>
### <span style="color:darkred">Write a short section that discusses the dataset stats (here you can recycle the work you did for Project Assignment A)</span>

To get an idea of the traffic situation in New York, what types of incidents, their outcome, what is the most common places for accidents to happen etc. we have done some different plotting of all collisions historically, per year, per borough and for specific vehicle types, times of day etc. Thus we hope to give you a better idea of the traffic scene in NYC.

The plots will start by describing basic summary statistics but will increase in complexity throughout the notebook. The plots will all be rendered using ``plotly`` and it's API in order to minimize the length of the notebook by exhibiting more information as mouse-over visuals directly in the plot itself.

The description associated with each plot will be positioned underneath the plots such that the reader can get a chance to make his/her own thoughts about the data before we bring our comments.

### <span style="color:darkred"> Inflicted boroughs</span>

In [9]:
# Count incidents in each borough
borough_cnt = ex.countSamples(traffic_data['BOROUGH'].replace(np.NaN, 'Unspecified'))

# Plot them
ex.createBarPlot(borough_cnt, 'Collisions per borough in all', 'Borough', 'Number of collisions', 'collisions_all', bgBorder, ticksAxes, 250)

From this plot it is easy to see that we are working with a lot of missing data (seen in bar "Unspecified"). If we however, overlook this for a while it looks as if Brooklyn is the borough where most accidents happen. Staten Island is by far the least inflicted borough, knowing it's geographical location and position relative to Manhattan/Brooklyn this makes good sense. Queens and Manhattan seem to have approximately the same number of accidents. 

We will look into whether this picture portrays the reality correct later with some geoplotting with ``geoplotlib``.

### <span style="color:darkred">Plotting the accidents</span>
Since we have a lot of missing information in the dataset, we chose to check whether the number of accidents would look the same plotted on a density map as it does when making the bar chart. 

Becuase of the high number of accidents each year, we have chosen only to plot a single month (May 2016) to give an idea of how 

In [13]:
year = 2016
yearData = traffic_data.loc[traffic_data['YEAR'].isin([year])]
oneMonth = yearData.loc[yearData['MONTH'].isin([5])] 
# Plot inline 
ex.plotGeoData(oneMonth, 'MONTH', "Current month")

Current month: 5
Samples: 20052
('smallest non-zero count', 1.7910636084771928e-08)
('max count:', 3.555695353321771)


Now, looking at the heat map of the accidents, the picture does not show that it is in fact Brooklyn that has the highest amount of accidents. If we look at Manhattan, the number of accidents is clearly higher here. This plot can make up for some of the missing values that are found for the boroughs and shows that Manhattan is actually "a bigger sinner" than both Queens and Brooklyn when it comes to accidents.

In [16]:
# Saving geoplot images to use on webpage
year = 2012
yearData = traffic_data.loc[traffic_data['YEAR'].isin([year])]
#ex.saveGeoData(yearData.loc[yearData['CONTRIBUTING FACTOR VEHICLE 1'].isin(['Driver Inattention/Distraction'])], 'MONTH', "Current month", str(year)[2:])
ex.saveGeoData(yearData, 'MONTH', "Current month", str(year)[2:])

Current month: 11
Samples: 15888
('smallest non-zero count', 1.7910636084771928e-08)
('max count:', 4.4128054383942352)
Current month: 10
Samples: 16861
('smallest non-zero count', 1.7910636084771928e-08)
('max count:', 3.0090484602706904)
Current month: 12
Samples: 17119
('smallest non-zero count', 1.7910636084771928e-08)
('max count:', 2.8426840225271008)
Current month: 9
Samples: 16532
('smallest non-zero count', 1.7910636084771928e-08)
('max count:', 3.2003116722806988)
Current month: 8
Samples: 17139
('smallest non-zero count', 1.7910636084771928e-08)
('max count:', 3.9328953814849217)
Current month: 7
Samples: 16988
('smallest non-zero count', 1.7910636084771928e-08)
('max count:', 3.1174848381808618)


### <span style="color:darkred">Primary vehicle in collision - per vehicle type</span>
In order to look into which types of vehicles that are most commonly involved in accidents, we have plottet all the vehicles involved in accidents for all the data.

In [57]:
# Replace NaN with 'unknown'
vehicle_types = traffic_data['VEHICLE TYPE CODE 1'].replace(np.NaN, 'OTHER')

# Count key/value pairs
collision_count = ex.countSamples(vehicle_types)

# createBarPlot(valueDict, plotTitle, xtitle, ytitle, saveName, bgBorder, ticksAxes, marginValue):
vehicles = ex.createBarPlot(collision_count, 'Primary vehicle in collison, NY 2012-2017', 'Vehicle type', 'Number of incidents', 'vechicle2', bgBorder, ticksAxes, 200)
vehicles

Passenger vehicles and sport/utility wagons are by far the two vehicle types that are most commonly involved in accidents. Out of the total of $1.007.130$ accidents, passenger vehicles are involved in more than $50\%$ of the accidents while the sport/utility wagon only account less than half of that. This visualization is also added on the webpage and will be described in further detail there.

In [45]:
# Save as json object for website
with open('primary_vehicle.json', 'w') as fp:
    json.dump(collision_count.most_common(), fp)

### <span style="color:darkred">Vehicle collisions - per contributing factor</span>

In [58]:
# Replace NaN with 'Unspecified'
factor_types = traffic_data['CONTRIBUTING FACTOR VEHICLE 1'].replace(np.NaN, 'Unspecified')

# Count number of occurences of the different categories
factor_count = ex.countSamples(factor_types)

# Remove unspecified in order to get a better picture of remaining entries
del factor_count['Unspecified']

# Save as json object for website
with open('factor_data.json', 'w') as fp:
    json.dump(factor_count.most_common(25), fp)

# createBarPlot(valueDict, plotTitle, xtitle, ytitle, saveName, bgBorder, ticksAxes, marginValue):
factors = ex.createBarPlot(factor_count, 'Contributing factors for collisions, NY 2012-2017', 'Reason/factor', 'Number of incidents', 'factors', bgBorder, ticksAxes, 250)
factors

Both the primary vehicle in collison and the contributiog factors are included as visualization on the final webpage. These two summary statistics show some of the basic facts about the different traffic accidents in New York and gives us a chance to show how to work with switching between different datasets in the same visualization, how to create the initial bar chart and update it with completely new values for both axes. We think this is a good way to show that we have understood the basic functionalities within the d3 visualization techniques when creating bar chart while it also gives the reader some intial information about the traffic in NYC.


### <span style="color:darkred"> Plotting the collisions per year</span>
Here, only data in range $2013-2016$ is used as the rest is incomple and will distort the overall picture as the data is plotted per year. in all, 10 vehicle typs are included.

In [62]:
# Create 2 X 5 grid for the 10 collision subplots
# Initialize variables
vehicle_types = ['PASSENGER VEHICLE', 'SPORT UTILITY / STATION WAGON',
                'VAN','BUS','SMALL COM VEH(4 TIRES)', 'LIVERY VEHICLE', 'MOTORCYCLE',
                 'PICK-UP TRUCK', 'LARGE COM VEH(6 OR MORE TIRES)', 'TAXI']
y_cnt = Counter()
i = 1 # plotting variable keeping track of column
k = 1 # plotting variable keeping track of row
j = 1 # plotting variable keeping track of axis number

fig = tools.make_subplots(rows=5, cols=2, subplot_titles=(vehicle_types))

# Run through each vehicle type
for vehicle in vehicle_types:
    # Make data structure with one collision category at a time
    temp_values = traffic_data.loc[traffic_data['VEHICLE TYPE CODE 1'].isin([vehicle])]
    # Add column with year
    for year in [2013, 2014, 2015, 2016]:
            y_cnt[year] = len(temp_values.loc[temp_values['YEAR'].isin([year])])
    
    trace = go.Bar(
    x=y_cnt.keys(),
    y=y_cnt.values()
    )
    fig.append_trace(trace, k, i)
    fig['layout']['yaxis'+str(j)].update(title='Number of incidents', range=[0, max(y_cnt.values())])
    
    if(i < 2):
        i += 1
    else:
        i = 1
        k += 1
    j += 1
    
fig['layout'].update(height=1200, width=1000, showlegend=False, title="Accidents per vehicle type 2013-2016", margin=dict(b=200))
py.iplot(fig, filename='subplot_vehicles')

This is the format of your plot grid:
[ (1,1) x1,y1 ]    [ (1,2) x2,y2 ]  
[ (2,1) x3,y3 ]    [ (2,2) x4,y4 ]  
[ (3,1) x5,y5 ]    [ (3,2) x6,y6 ]  
[ (4,1) x7,y7 ]    [ (4,2) x8,y8 ]  
[ (5,1) x9,y9 ]    [ (5,2) x10,y10 ]



As in our previous assignments we have plotted the number of accidents for each year for some of the most common vehicle types involved in accidents. There are several patterns in the vehicle specific development of accidents from $2013-2016$. It looks as if the larger vehicles have had a decrease in number of accidents in 2016 compared to earlier years, whereas the number of accidents for passenger vehicles and motor cycles have increased. 

### <span style="color:darkred"> Number of traffic deaths</span>

In [17]:
persons_killed = traffic_data['NUMBER OF PERSONS KILLED'].replace(np.NaN, 'OTHER')
kill_count = {}
for val in traffic_data['YEAR'].unique():
    if val != np.int64(2017) and val != np.int64(2012):
        temp = traffic_data.loc[traffic_data['YEAR'].isin([val])]
        kill_count[val] = int(temp['NUMBER OF PERSONS KILLED'].sum(axis=0))

#killed = ex.createBarPlot(kill_count, 'Number of persons killed per year, 2012-2017', 'Year', 'Number of deaths', 'deaths_all', bgBorder, ticksAxes, 150)
#killed
killed = ex.createXYBarPlot(kill_count.keys(), kill_count.values(), 'Number of persons killed per year in NYC, 2012-2017', 'Year', 'Number of deaths', 'deaths_all', bgBorder, ticksAxes, 150)
killed

In [21]:
for years in [2012, 2013, 2014, 2015, 2016, 2017]:
    print years, len(traffic_data.loc[traffic_data['YEAR'].isin([years])])

2012 100527
2013 203715
2014 205960
2015 217636
2016 227658
2017 51634


### <span style="color:darkred">Involved parties</span>
* [Subplot documentation for Plotly](https://plot.ly/python/subplots/)

#### A slowly decreasing trend

In [24]:
# Extracting needed information
focus_group = ['NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST KILLED']
focus = {}

# Creating initial figure for subplots
fig = tools.make_subplots(rows=1, cols=4, subplot_titles=("2013", "2014", "2015", "2016"))

i = 1 # Keeping track of plot grid columns
k = 1 # Setting axis range for all y axes 
    
for year in years_of_interest:
    for group in focus_group:
        yearly_data = traffic_data.loc[traffic_data['YEAR'].isin([year])]
        focus[group] = yearly_data[group].sum()
    
    trace = go.Bar(
    x=focus.keys(),
    y=focus.values(),
    name=year
    )
    # Add plot to figure
    fig.append_trace(trace, 1, i)
    fig['layout']['yaxis'+str(k)].update(title='Year', range=[0, 180])
    
    i += 1 
    k += 1

# Plot result
fig['layout'].update(height=600, width=1100, showlegend=False, title='Casualties per year in NYC traffic accidents', margin=dict(b=200))
py.image.save_as(fig, filename='subplots-deaths.png')
py.iplot(fig, filename='simple-subplot')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]  [ (1,3) x3,y3 ]  [ (1,4) x4,y4 ]



Number of people killed in traffic accidents in New York is slowly decreasing.


### <span style="color:darkred">Are the incidents prone to happen at a specific time of day?</span>

In [8]:
bor = ["Unknown", 'BROOKLYN', 'BRONX', 'MANHATTAN', 'QUEENS', 'STATEN ISLAND']
n = ex.plotBarInSubplotGrid(2, 3, bor, "Incidents in NYC per hour", traffic_data, 'BOROUGH', 'HOUR', 'hourly', 21000, 1000, 1200)
n

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]  [ (1,3) x3,y3 ]
[ (2,1) x4,y4 ]  [ (2,2) x5,y5 ]  [ (2,3) x6,y6 ]



In [24]:
# Saving number of accidents per borough per year to json object for website
hour_dict = {}
for borough in bor:
    wrk = traffic_data.loc[traffic_data['BOROUGH'].isin([borough])]
    # Count incidents in each borough
    borough_cnt = countSamples(wrk['HOUR'])
    hour_dict[borough] = dict(borough_cnt.most_common())

# Save data structure to json object
with open('hour_data.json', 'w') as fp:
    json.dump(hour_dict, fp)

AttributeError: 'list' object has no attribute 'most_common'

It looks like New York acutally sleeps -- for an hour! Of course, this is an overall picture and we cannot draw many conclusions based on this. Yes, most accidents happens between 8AM and 8PM, but, it would be better to look at the individual boroughs, wouldn't it?

Lets check that out!
From looking at this we cannot say more about the trend. We can see that most accidents happens in Queens and Brooklyn, but besides this, the times in which the majority of the accidents happen are the same, except from the fact that Manhatten, unlike the others do not have the "morning peak" from $8-9$.

### <span style="color:darkred">Accidents according to time of year -- does it have any impact?</span>
<span style="color:red">Should we include this?</span>
We have actively chosen to remove all incidents without any geolocation in order to plot the results more correctly.

In [22]:
# Finding all incidents were people got killed as a result of the collisions
bor2016 = traffic_data.loc[traffic_data['YEAR'].isin([2016])]
borough_cnt = ex.countSamples(bor2016['BOROUGH'])

# Plot count per borough
bor = ex.createXYBarPlot(borough_cnt.keys(), borough_cnt.values(), "Accidents per borough in 2016", 'Borough', 'Number of person killed', 'accidents_per_borough_2016', bgBorder, ticksAxes, 200)
bor

### <span style="color:darkred">What can we use this for?</span>
Looking at the geodata, we can see that the areas most commonly exposed to traffic accidents are Midtown (around Times Square),  East Village and Lower Manhattan along with Brooklyn Heights and around Flushing Meadows Corona Park. Which in fact changes the picture of where there are most accidents. From looking solely at the number of accidents in each borough, we had a lot of missing data, but it seems as though a lot of the missing data belongs to Manhattan, which can also be seen from the heatmap that clearly shows Manhattan as the area most exposed to traffic accidents!

Is this because it would be better (imagewise) for NYC that one of the lesser visited boroughs had the highest number of crimes?

#### <span style="color:darkred">So the funny thing is..</span>
Looking at the plots, somehow, we get the feeling that in August and September there are less traffic accidents, but **this is wrong** actually, there are just as many but for both months more than $84\%$ of the traffic accidents **are not properly reported** -- at least all the geodata info is missing! Is this because a lot of accidents attract less tourists in the summer? 

Take a look below:

----
## <span style="color:darkred">The Demographics of New York</span>

In [16]:
# Demographics of New York 
ny_demographics = pd.read_excel('NY_demographics.xlsx')
ny_demographics

,borough,population,sqMi,sqKm,personsPerSqMi,personsPrSqKm
0,Manhattan,1644518,22.83,59.1,72033.00,27826
1,Bronx,1455444,42.00,110.0,34653.00,13231
2,Brooklyn,2636735,71.00,180.0,37137.00,14649
3,Queens,2339150,109.00,280.0,21.46,8354
4,Staten Island,474558,58.50,152.0,8112.00,3132


Source: [Demographics of new York - Wikipedia](https://en.wikipedia.org/wiki/Demographics_of_New_York_City)

* Number of fatalities/population
* Basic plotting/statistics

<span style="color:red">Can we do some more with this? - I was thinking using it as underlying choropleth layer on shapefile?</span>

In [17]:
# Find number of people per square kilometer for each borough in NYC
i = 0
for values in ['BRONX', 'BROOKLYN', 'MANHATTAN', 'QUEENS', 'STATEN ISLAND']:
    print values, float(borough_cnt['BRONX'])/ny_demographics.iloc[i,1]
    i += 1

BRONX 0.0131850183458
BROOKLYN 0.0148978593474
MANHATTAN 0.00822342783784
QUEENS 0.00926960648099
STATEN ISLAND 0.0456909376725


In [18]:
# Plot person per square kilometer
ppl_pr_sqkm = ex.createXYBarPlot(ny_demographics['borough'], ny_demographics['personsPrSqKm'], "Number of people per square kilometer", 'Borough', 'Number of people', 'ppl_pr_sqkm', bgBorder, ticksAxes, 100)
ppl_pr_sqkm

From the above plot it is made clear that indeed Manhattan has a great deal of people per square kilometer. **Manhattan has in fact more than double the amount of people compared any of the neighbouring boroughs**. Naturally one should expect more incidents here, since more people makes for a more activity both on the sidewalks and in the streets. One thing one also have to take into account is the high number of people visitng Manhattan which without a doubt will make the number of people per square kilometer even higher.

----
## <span style="color:darkred">Theory - theoretical tools</span>

### <span style="color:darkred">Machine learning tools and why tools chosen are right for our problem.</span>
* **K-means**: We chose to cluster our data using K-means, as we wanted to know how the distribution of traffic accidents in NYC changed over the years. Since K-means is an unsupervised machine learning method it is a good choice to use when exploring data. Thus we decided not use any target data for the model. We chose data from may in 2013-2016 as the whole dataset contained too many observations for the clustering to run smoothly on the website.
* **Decision tree**: We chose to use decision trees to predict the causes behind the accidents. We wanted to use more than just geolocation to predict on our data and a decision tree seemed fitting as it can use both categorical aswell as continous features at the same time. Furthermore it is also tool which can be easily visualized and understood by the reader. Decision trees can be used to find hidden


### <span style="color:darkred">Model selection and splitting of data</span>
* **K-means**: For clustering we didn't need to split the data into test and training sets, since we opted to model the data without targets. This decision also means that we didn't need (and couldn't use) cross validation to test our model preformance.
* **Decision tree**: To later be able to test the preformance of the decision tree classifier we decided to split the data into a training and testing set. This split consited accordingly of 90 % and 10 % of the data with the use of the function train_test_split from the scikitlearn library. For the decision tree we decided to reduce the number of targets from the original 49 to the 10 most common factors. In regards to the features wetried using street name as a variable, but this decision resulted in gross overfitting. But why? The answer is simple. The feature of street names contain too many different categories as there of course are a lot of different streets in NYC. The model would make too many very specific rulesto fit the training data perfectly. Thus we decided to drop the feature from the model.


### <span style="color:darkred">Model performance</span>
* **K-means**: The lack of target data again means that model performance can't be done.
* **Decision tree**: From splitting the dataset into training and test data we are able to measure preformance on the model on both the data subsets. This way we were able to see if our model overfitted to the training data. The preformance measure we used was just to see how big a percent of the datasets we predicted correctly. The final model we produced had a training and testing error of around 35.5 % which is not great but still better than random, which would be around 10 % since we have 10 different targets.


## <span style="color:darkred">Solutions for the machine learning methods</span>

### <span style="color:darkred">K-means clustering</span>

In [2]:
# Importing data set (used for both machine learning methods)
filename = 'Traffic_data.csv'
traffic_data = pd.read_csv(filename, low_memory=False)
traffic_data

,Unnamed: 0,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,...,UNIQUE KEY,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,YEAR,MONTH,HOUR,WEEKDAY
0,0,03/31/2017,0:00,NaN,NaN,40.645615,-73.909900,"(40.645615, -73.9099)",FOSTER AVENUE,NaN,...,3643404,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN,2017,3,0,4
1,1,03/31/2017,0:00,NaN,NaN,40.762737,-73.839510,"(40.762737, -73.83951)",NaN,NaN,...,3643942,SPORT UTILITY / STATION WAGON,PASSENGER VEHICLE,NaN,NaN,NaN,2017,3,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007128,1007128,07/01/2012,9:57,MANHATTAN,10065,40.765242,-73.957868,"(40.7652424, -73.9578679)",1 AVENUE,EAST 68 STREET,...,44907,PASSENGER VEHICLE,TAXI,NaN,NaN,NaN,2012,7,9,6
1007129,1007129,07/01/2012,9:59,BRONX,10452,40.835397,-73.920305,"(40.835397, -73.920305)",EAST 167 STREET,GERARD AVENUE,...,85154,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN,2012,7,9,6


In [118]:
# Extracting coordinate data
coordinates  = traffic_data[['LATITUDE','LONGITUDE','YEAR']].dropna(axis = 0, how = 'any')
focus_years  = [2013,2014,2015,2016]

# Number of clusters to use
n_clusters   = 4

# Initializing dictionaries to store data
cluster_data = {}
centroids    = {}

for year in focus_years:
    data = coordinates.loc[coordinates['YEAR'].isin([year])]
    
    # Train KMeans on data
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(data[['LATITUDE','LONGITUDE']])
    
    # Save the centroids to an array
    centroid = kmeans.cluster_centers_
    centroid_array = []
    
    for j in range(0,n_clusters):
        centroid_cord = {}
        centroid_cord['LATITUDE'] = centroid[j][0]
        centroid_cord['LONGITUDE'] = centroid[j][1]
        centroid_array.append(centroid_cord)
    centroids[str(year)] = centroid_array
    
    data['CLUSTER'] = kmeans.predict(data[['LATITUDE','LONGITUDE']])
    
    cluster_data[str(year)] = data[['LATITUDE','LONGITUDE','CLUSTER']].to_dict('list')

model_data = {}
model_data['centroids'] = centroids
model_data['datapoints'] = cluster_data

In [119]:
# Write JSON file with datapoints
with io.open('model_data.json', 'w', encoding='utf8') as outfile:
    str_ = json.dumps(model_data,
                      indent=4, sort_keys=True,
                      separators=(',', ':'), ensure_ascii=False)
    outfile.write(to_unicode(str_))

### <span style="color:darkred">Decision Tree</span>

In [3]:
# Duplicating data 
features  = ['CONTRIBUTING FACTOR VEHICLE 1','BOROUGH','NUMBER OF PERSONS KILLED','VEHICLE TYPE CODE 1',
            'HOUR','WEEKDAY']
tree_data = traffic_data[features]

# Replacing NA's with 'OTHER' for vehicle type
tree_data['VEHICLE TYPE CODE 1'].fillna('OTHER', inplace=True)

tree_data = tree_data[tree_data['CONTRIBUTING FACTOR VEHICLE 1'] != 'Unspecified']
tree_data = tree_data[tree_data['BOROUGH'] != 'UNKNOWN']
tree_data = tree_data.dropna(axis = 0, how = 'any')

# Remove uncommon contributing factors (as shown in plot of factors)
common_factors = ['Driver Inattention/Distraction','Fatigued/Drowsy','Failure to Yield Right-of-Way',
                  'Other Vehicular','Backing Unsafely','Turning Improperly','Lost Consciousness','Prescription Medication',
                  'Traffic Control Disregarded','Driver Inexperience']

print len(tree_data)
tree_data = tree_data.loc[tree_data['CONTRIBUTING FACTOR VEHICLE 1'].isin(common_factors)]
print len(tree_data)

334947
264187


In [4]:
X = tree_data[features[1:]]
# Encode input values as an enumerated type or categorical variable
X.loc[:,'BOROUGH']                  = pd.factorize(X['BOROUGH'])[0]
#X.loc[:,'ON STREET NAME']          = pd.factorize(X['ON STREET NAME'])[0]
X.loc[:,'NUMBER OF PERSONS KILLED'] = pd.factorize(X['NUMBER OF PERSONS KILLED'])[0]
X.loc[:,'VEHICLE TYPE CODE 1']      = pd.factorize(X['VEHICLE TYPE CODE 1'])[0]
X.loc[:,'HOUR']                     = pd.factorize(X['HOUR'])[0]
X.loc[:,'WEEKDAY']                  = pd.factorize(X['WEEKDAY'])[0]

y = pd.factorize(tree_data[features[0]])[0]

# Splitting testing and training data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

In [5]:
# Model training and prediction
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
test_pred = clf.predict(X_test)
train_pred = clf.predict(X_train)

print 'The test score is:    ', ex.score(test_pred,y_test)
print 'The training score is:', ex.score(train_pred,y_train)

The test score is:     0.341307392407
The training score is: 0.371113858888


----
## <span style="color:darkred">Visualizations</span>

<span style="color:red"> Should we chose something different on the webpage?</span>

### <span style="color:darkred">Explain the visualizations you've chosen.</span>


### <span style="color:darkred">Why are they right for the story you want to tell?</span>

----
## <span style="color:darkred">Discussion - critical thinking</span>


### <span style="color:darkred">What went well?</span>
* <span style="color:red">Model selection and prediction, right?</span>
* Set-up/design of webpage 

### <span style="color:darkred">What is still missing? What could be improved?, Why?</span>
#### <span style="color:darkred">Data-wise</span>
Need to write more about

* <span style="color:red">Combining different datasets</span>
* <span style="color:red">Adding information about traffic density</span>
* <span style="color:red">Size of cluster file (but do include a lot of data</span>

We really wanted to do some more work on the missing values in the dataset. We found the module ``geopy`` which lets you write an address and get corresponding geo location or the other way around, write geo location and get street information such as Street name, number, state, country etc. an example of both is given below.

The problem however is that the API that is called to get the geo locations/address values only allow a minimum of calls per day and with a total of $32.396$ values for just location, this has been impossible to acheive. However we still wanted to mention it as this was our intention and probably would have been easy to do had we had a developer account. 

#### <span style="color:darkred">Machine Learning</span>


#### <span style="color:darkred"> Visualizations</span>
* Dived into specific accidents over time, llok at only a year? Find patterns.

#### <span style="color:darkred"> Overall in the solution</span>

## <span style="color:darkred">References</span>
In relation to traffic in New York City, we have found a few sources describing the traffic in New York and some previous visualizations:
#### <span style="color:darkred">Related articles and inspiration</span>

<span style="color:red">Need to write these more academically correct!</span>

* Decreasing number of [Traffic deaths in NYC](https://www.nytimes.com/2016/01/02/nyregion/number-of-traffic-deaths-in-new-york-falls-for-a-second-year-in-a-row.html?_r=0)
* [**Viewing.nyc's**](https://viewing.nyc/) really got it!
  * [Visualization of NYC traffic data](https://viewing.nyc/see-wazes-hypnotic-visualization-of-new-york-city-traffic-patterns/)
  * [Distracted driving in New York](https://viewing.nyc/heres-a-scary-map-of-all-distracted-driving-collisions-in-new-york-city/)
* [ArcGIS Population/Traffic](https://www.arcgis.com/home/webmap/viewer.html?webmap=df41300d3d4344baa080c86349bfd59b)
* [YouTube-inspiration](https://www.youtube.com/watch?v=QFv7AFRmmTs)
* [Blocks.org - Bar charts](http://bl.ocks.org/juan-cb/faf62e91e3c70a99a306)

* [Vision Zero - basic information](nyc.gov/visionzero)
* [Vision Zero - accident map](nyc.gov/visionzero)
This map has been created in relation to Vision Zero and has a lot of information about speed limits, where accidents happen (year-by-year), risk areas, number of accidents per precinct etc.
* [Scrolling as opposed to clicking!](https://bost.ocks.org/mike/scroll/)
* [Shapefile](http://www1.nyc.gov/site/planning/data-maps/open-data/districts-download-metadata.page)
* [Choropleth map](https://bl.ocks.org/mbostock/4060606)
* [Switching between images](http://stackoverflow.com/questions/13975891/change-image-in-html-page-every-few-seconds)